In [1]:
# Pull yfinance price data for listings in the tsx constituent db table
# Import packages
import yfinance as yf
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

# Import utilities
from etl_utils import extract_query, load_query

In [4]:
load_dotenv()

# Create db connection string
db_username = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_conn_str = f"mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/tsx_composite_index"

In [38]:
# Import current constituent list - get both ticker and id
import_query = """
SELECT yfinance_ticker, company_id
FROM ticker_history
WHERE end_date IS NULL
"""

ticker_list = extract_query(sql_query=import_query, db_conn_str=db_conn_str)

Connection Successful!
SQL script executed successfully!


In [27]:
# Set start date and end date - These would be set by airflow
start_date = input("Enter start date (yyyy-mm-dd): ")
end_date = input("Enter end date (yyyy-mm-dd): ")

In [ ]:
# Iterate over the dataframe and pull the price history for each ticker
# Simultaneously, we can update the industry/sector, which will be stored in a separate table
dfs_history = []
dfs_info = ticker_list

In [ ]:
for _, row in ticker_list.iterrows():
    # Pull history data
    ticker = yf.Ticker(row['yfinance_ticker'])
    df = ticker.history(start=start_date, end=end_date, interval="1d")

    # Format date
    df.reset_index(inplace = True)
    df['Date'] = df['Date'].dt.tz_localize(None)

    # Add columns for ticker and company_id
    df['yfinance_ticker'] = row['yfinance_ticker']
    df['company_id'] = row['company_id']

In [11]:
ticker = yf.Ticker("TD.TO")
ticker2 = yf.Ticker("ATD.TO")

In [12]:
fin1 = ticker.get_financials()
fin2 = ticker2.get_financials()

In [16]:
merged = fin1.merge(fin2, how = "outer", left_index = True, right_index = True)

In [30]:
df = ticker.history(start=start_date, end=end_date, interval="1d")

In [35]:
df.reset_index(inplace = True)

In [37]:
df['Date'] = df['Date'].dt.tz_localize(None)

0      2020-01-02
1      2020-01-03
2      2020-01-06
3      2020-01-07
4      2020-01-08
          ...    
1480   2025-11-24
1481   2025-11-25
1482   2025-11-26
1483   2025-11-27
1484   2025-11-28
Name: Date, Length: 1485, dtype: datetime64[ns]

In [14]:
hist.columns

MultiIndex([(       'Close', 'RY.TO'),
            (       'Close', 'TD.TO'),
            (   'Dividends', 'RY.TO'),
            (   'Dividends', 'TD.TO'),
            (        'High', 'RY.TO'),
            (        'High', 'TD.TO'),
            (         'Low', 'RY.TO'),
            (         'Low', 'TD.TO'),
            (        'Open', 'RY.TO'),
            (        'Open', 'TD.TO'),
            ('Stock Splits', 'RY.TO'),
            ('Stock Splits', 'TD.TO'),
            (      'Volume', 'RY.TO'),
            (      'Volume', 'TD.TO')],
           names=['Price', 'Ticker'])

In [9]:
ticker.history(start="2025-01-01", end="2025-11-30")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-01-02 00:00:00-05:00,73.480493,73.490041,72.879050,73.031799,18507000,0.0,0.0
2025-01-03 00:00:00-05:00,73.289549,74.301497,73.108167,74.253761,17948300,0.0,0.0
2025-01-06 00:00:00-05:00,74.769281,74.950669,74.167845,74.253761,22054800,0.0,0.0
2025-01-07 00:00:00-05:00,74.291953,75.141604,74.225127,74.922035,22302900,0.0,0.0
2025-01-08 00:00:00-05:00,75.093877,75.179794,74.196491,74.387421,23749600,0.0,0.0
...,...,...,...,...,...,...,...
2025-11-24 00:00:00-05:00,115.589996,116.930000,115.360001,116.709999,7345500,0.0,0.0
2025-11-25 00:00:00-05:00,116.709999,117.129997,115.599998,116.040001,4125500,0.0,0.0
2025-11-26 00:00:00-05:00,116.419998,117.330002,116.160004,117.150002,3689000,0.0,0.0


In [58]:
aem_hist.head(20)

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2025-10-27 09:30:00-04:00,218.580002,223.850006,218.309998,222.940002,57839,0.0,0.0
1,2025-10-27 10:00:00-04:00,222.750000,223.139999,215.830002,216.250000,255662,0.0,0.0
2,2025-10-27 10:30:00-04:00,216.580002,216.740005,214.000000,214.889999,310714,0.0,0.0
3,2025-10-27 11:00:00-04:00,214.720001,216.550003,213.009995,216.550003,272772,0.0,0.0
4,2025-10-27 11:30:00-04:00,216.559998,217.000000,214.929993,217.000000,206827,0.0,0.0
5,2025-10-27 12:00:00-04:00,217.000000,217.679993,216.100006,216.779999,225919,0.0,0.0
6,2025-10-27 12:30:00-04:00,216.779999,217.500000,216.580002,217.065002,126150,0.0,0.0
7,2025-10-27 13:00:00-04:00,217.115005,217.910004,217.000000,217.550003,50819,0.0,0.0
8,2025-10-27 13:30:00-04:00,217.554993,218.100006,217.300003,217.479996,69026,0.0,0.0
9,2025-10-27 14:00:00-04:00,217.509995,217.619995,216.500000,216.800003,143635,0.0,0.0
